In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

# Define the Kaggle credentials path
kaggle_json_path = "/content/drive/MyDrive/kaggle.json"

# Move kaggle.json to ~/.kaggle/
os.makedirs("/root/.kaggle", exist_ok=True)
os.system(f"cp {kaggle_json_path} /root/.kaggle/")

# Change permission
os.system("chmod 600 /root/.kaggle/kaggle.json")

# Now, you can download datasets using Kaggle API
!kaggle datasets download -d aryashah2k/indian-medicinal-leaves-dataset -p "/content/drive/My Drive/"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset URL: https://www.kaggle.com/datasets/aryashah2k/indian-medicinal-leaves-dataset
License(s): CC-BY-NC-SA-4.0
100% 9.00G/9.00G [05:47<00:00, 29.0MB/s]
100% 9.00G/9.00G [05:47<00:00, 27.8MB/s]


In [ ]:
import os
import shutil
import random

dataset_path = "/content/Indian Medicinal Leaves Image Datasets/Medicinal Leaf dataset"
train_path = "/content/Splitted Dataset Leaves/train"
val_path = "/content/Splitted Dataset Leaves/val"
test_path = "/content/Splitted Dataset Leaves/test"

# Create directories
for folder in [train_path, val_path, test_path]:
    os.makedirs(folder, exist_ok=True)

# Split the dataset into train (70%), val (15%), test (15%)
split_ratio = {'train': 0.7, 'val': 0.15, 'test': 0.15}

for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)

    if os.path.isdir(category_path):  # Only process folders
        images = os.listdir(category_path)
        random.shuffle(images)  # Shuffle for randomness

        train_size = int(len(images) * split_ratio['train'])
        val_size = int(len(images) * split_ratio['val'])

        train_images = images[:train_size]
        val_images = images[train_size:train_size + val_size]
        test_images = images[train_size + val_size:]

        for img_set, target_folder in zip([train_images, val_images, test_images], [train_path, val_path, test_path]):
            category_folder = os.path.join(target_folder, category)
            os.makedirs(category_folder, exist_ok=True)

            for img in img_set:
                src = os.path.join(category_path, img)
                dst = os.path.join(category_folder, img)
                shutil.copy(src, dst)

print("Dataset split into Train, Validation, and Test sets.")


Dataset split into Train, Validation, and Test sets.


In [ ]:
import zipfile
import os

# Define dataset path (assuming it's in Drive)
dataset_path = "/content/drive/MyDrive/indian-medicinal-leaves-dataset.zip"
extract_to = "/content/"  # Extract to Colab's local storage

# Extract the ZIP file
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Dataset extracted successfully at /content/")


Dataset extracted successfully at /content/


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG19

In [ ]:
# Paths to dataset folders
TRAIN_DIR = '/content/Splitted Dataset Leaves/train'
TEST_DIR = '/content/Splitted Dataset Leaves/test'
VAL_DIR = '/content/Splitted Dataset Leaves/val'

# Image size and batch size
IMG_SIZE = (256, 256)  # Resize all images to 256X256
BATCH_SIZE = 32


train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'   # categorical
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  #categorical
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  #categorical
)


Found 4794 images belonging to 80 classes.
Found 1105 images belonging to 80 classes.
Found 1001 images belonging to 80 classes.


In [ ]:
'''def build_model():
    model = Sequential([
        Conv2D(128, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(512, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(80, activation='softmax')  # softmax for multi classification
    ])

    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',  #sparse_categorical_crossentropy
                  metrics=['accuracy'])
    return model


model = build_model()
model.summary()'''

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 254, 254, 128)       │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 127, 127, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 125, 125, 256)       │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 62, 62, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 60, 60, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 30, 30, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 460800)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 512)                 │     235,930,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 80)                  │          41,040 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 237,450,064 (905.80 MB)

 Trainable params: 237,450,064 (905.80 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Load VGG19 pre-trained model (excluding top layers)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))


base_model.trainable = False

# Create the model
model = Sequential([
    base_model,  # VGG19 base model
    MaxPooling2D(),
    Flatten(),  # Flatten the output
    Dense(512, activation='relu'),  # Fully connected layer
    Dropout(0.2),  # Dropout to prevent overfitting
    Dense(80, activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator

)

Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.1324 - loss: 3.7832 - val_accuracy: 0.3986 - val_loss: 2.5958
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 264s 2s/step - accuracy: 0.4093 - loss: 2.3605 - val_accuracy: 0.5465 - val_loss: 1.7908
Epoch 3/10
 62/150 ━━━━━━━━━━━━━━━━━━━━ 2:01 1s/step - accuracy: 0.5645 - loss: 1.6365

In [ ]:
model.save('temp.keras')

In [ ]:
import requests

# Replace with your actual Perenual API key
API_KEY = "sk-DQkM67909b83bd9c78328"

def get_plant_id(plant_name):
    """Search for a plant by name and return its ID"""
    url = f"https://perenual.com/api/species-list?key={API_KEY}&q={plant_name}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if "data" in data and data["data"]:
            plant_id = data["data"][0]["id"]  # Get the first matching plant ID
            return plant_id
        else:
            print("No plant found with that name.")
            return None
    else:
        print("Error fetching plant data:", response.status_code)
        return None

def get_plant_details(plant_id):
    """Fetch detailed information about a plant using its ID"""
    url = f"https://perenual.com/api/species/details/{plant_id}?key={API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()  # Return the plant details as JSON
    else:
        print("Error fetching plant details:", response.status_code)
        return None

# Example Usage
plant_name = "Aloe Vera"
plant_id = get_plant_id(plant_name)

if plant_id:
    details = get_plant_details(plant_id)
    if details:
        print("Plant Name:", details.get("common_name", "N/A"))
        print("Scientific Name:", details.get("scientific_name", "N/A"))
        print("Description:", details.get("description", "No description available"))


Plant Name: aloe
Scientific Name: ['Aloe vera']
Description: Aloe vera is a fantastic plant species known for its many amazing health benefits. Consisting of thick leaves with a gel-like interior, aloe vera has long been used as a traditional remedy for treating skin conditions. It is a great source of vitamins, minerals, and amino acids, making it a great natural remedy for common ailments. Aloe vera is also incredibly versatile, with the gel and sap being used for everything from digestive disorders to eye health. It is even believed to reduce inflammation and improve circulation. With its myriad of benefits, aloe vera is definitely an amazing plant species.
